# Creating Custom Agents using LangChain Framework

In [142]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import tool 
from langchain.prompts import PromptTemplate
from langchain_core.tools import render_text_description
from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser
from typing import Union,List,Tuple
from langchain_core.agents import AgentAction,AgentFinish

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### Defining Tool

In [4]:
@tool
def get_text_length(text:str)->int:
    """Return the length of a text by characters"""
    print(f"get_text_length enter with {text=}")
    text = text.strip("'\n").strip('"')
    return len(text)

In [5]:
result = get_text_length.invoke('hello')
print(result)

get_text_length enter with text='hello'
5


In [44]:
tools = [get_text_length]
print(tools)

[StructuredTool(name='get_text_length', description='Return the length of a text by characters', args_schema=<class 'pydantic.v1.main.get_text_lengthSchema'>, func=<function get_text_length at 0x000001DFA6AADEE0>)]


### Template 

In [157]:
template = """ 
Answer the following questions as best you can. You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:
"""

In [56]:
# prompt = PromptTemplate.from_template(template='say {foo}')
prompt = PromptTemplate(template="say {greet} {par_var}", input_variables=["greet"],partial_variables={"par_var":'kiran'})
prompt.format(greet='yoyo',par_var="bar")

prompt.dict()
# print(prompt) promptValue = {""}

# result=prompt.invoke({"foo":"hello"})
# print(result.text)

{'name': None,
 'input_variables': ['greet'],
 'input_types': {},
 'output_parser': None,
 'partial_variables': {'par_var': 'kiran'},
 'metadata': None,
 'tags': None,
 'template': 'say {greet} {par_var}',
 'template_format': 'f-string',
 'validate_template': False,
 '_type': 'prompt'}

In [68]:
 promptValue = {"greet": lambda x:x["greet"]} | prompt

promptValue.invoke({"greet":"hello"})
# print(promptValue)

StringPromptValue(text='say hello kiran')

In [42]:
# List of integers
numbers = [1, 2, 3, 4, 5]

# Convert each integer to a string
str_numbers = map(str, numbers)

# Join the string representations with commas
result = '*'.join(str_numbers)

# Output the result
print(result)

1*2*3*4*5


In [53]:
render_text_description(tools)

'get_text_length(text: str) -> int - Return the length of a text by characters'

In [160]:
prompt = PromptTemplate \
.from_template(template=template) \
.partial(tools=render_text_description(tools),tool_names=', '.join([t.name for t in tools]))  # LLm only accepts STRING as input

In [200]:
# llm = ChatOpenAI(temperature=0,model_kwargs={'stop':["\nObservation","Observation"]})
llm = ChatOpenAI(temperature=0,model_kwargs={'stop':["\nObservation","Observation"]})

In [162]:
llm.invoke('calculate no.of characters in word DOG')

AIMessage(content='The word "DOG" has 3 characters.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 15, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-18561819-5519-4279-94c9-5a7450accc20-0', usage_metadata={'input_tokens': 15, 'output_tokens': 10, 'total_tokens': 25})

In [ ]:
promptValue = {"greet": lambda x:x["greet"]} | prompt
result  = promptValue.invoke({"greet":"hello"})
result.text

## Creating AGENT

### Without OUT-PARSER 

In [ ]:
agent = {"input":lambda x:x["input"]} | prompt | llm 
res = agent.invoke({"input":"what is the text length of 'dog' in characters?"})
print(res)

### With React_style_output_parser 

In [163]:
 agent = {"input":lambda x:x["input"]} | prompt | llm | ReActSingleInputOutputParser()
          #  |                |
          # ^to prompt :      ^to Dictionary form invoke


In [164]:
res = agent.invoke({"input":"what is the text length of 'dog' in characters?"})
print(res)

tool='get_text_length' tool_input="'dog'" log="I should use the get_text_length function to determine the length of the text 'dog'.\nAction: get_text_length\nAction Input: 'dog'"


`Till now the LLM Figured out what tool is best for the task/question`

## Execution of tool

In [ ]:
agent_step:Union[AgentAction,AgentFinish] = agent.invoke({"input":"What is the length of 'fish in chracters?"})
print(agent_step)

In [121]:
isinstance(agent_step,AgentAction)

True

In [122]:
agent_step.dict()

{'tool': 'get_text_length',
 'tool_input': 'fish',
 'log': 'I should use the get_text_length function to determine the length of the word "fish".\nAction: get_text_length\nAction Input: "fish"',
 'type': 'AgentAction'}

In [123]:
def find_tool_by_name(tools,tool_name):
    for tool in tools:
        if tool.name == tool_name:
            return tool


find_tool_by_name(tools,agent_step.tool).dict()

{'name': 'get_text_length',
 'description': 'Return the length of a text by characters',
 'args_schema': pydantic.v1.main.get_text_lengthSchema,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'func': <function __main__.get_text_length(text: str) -> int>,
 'coroutine': None}

### How does a AGENT EXECUTOR Works form inside?

In [124]:
if(isinstance(agent_step,AgentAction)):
    tool_name   = agent_step.tool
    tool_to_use = find_tool_by_name(tools,tool_name).func
    tool_input  = agent_step.tool_input
    observation = tool_to_use(tool_input)
    print(f"{observation=}")

get_text_length enter with text='fish'
observation=4


In [125]:
len(tool_input)

4

## Agent with scratchpad (History of chats/observations)

In [197]:
intermediate_steps=[]

In [134]:
template = """ 
Answer the following questions as best you can. You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad} 
"""

In [165]:
agent_step:Union[AgentAction,AgentFinish] = agent.invoke({"input":"what is the text length of dog in characters?","agent_scratchpad":intermediate_steps})

print(agent_step)

tool='get_text_length' tool_input='dog' log='I should use the get_text_length function to find the length of the text "dog".\nAction: get_text_length\nAction Input: "dog"'


In [201]:
agent = {
    "input":lambda x:x["input"],"agent_scratchpad":lambda x:format_log_to_str(x["agent_scratchpad"])
    } | prompt | llm | ReActSingleInputOutputParser()

agent_step:Union[AgentAction,AgentFinish] = agent.invoke({"input":"what is the text length of dog in characters?","agent_scratchpad":intermediate_steps})

if(isinstance(agent_step,AgentAction)):
    tool_name   = agent_step.tool
    tool_to_use = find_tool_by_name(tools,tool_name).func
    tool_input  = agent_step.tool_input
    
    observation = tool_to_use(tool_input)
    print(f"{observation=}")
    intermediate_steps.append((agent_step,observation))
    
# print(intermediate_steps)

print(agent_step)


get_text_length enter with text='dog'
observation=3
tool='get_text_length' tool_input='dog' log='I should use the get_text_length function to determine the length of the text "dog".\nAction: get_text_length\nAction Input: "dog"'


In [202]:
agent_step.dict()

{'tool': 'get_text_length',
 'tool_input': 'dog',
 'log': 'I should use the get_text_length function to determine the length of the text "dog".\nAction: get_text_length\nAction Input: "dog"',
 'type': 'AgentAction'}

In [184]:
def format_log_to_str(intermediate_steps:List[Tuple[AgentAction,str]],observation_prefix:str="Observation:",llm_prefix:str="Thought:") -> str:
    """Construct the scratchpad that lets the agent continue its thought process."""
    thoughts=""
    for action , observation in intermediate_steps:
        thoughts += action.log
        thoughts += f"\n{observation_prefix}{observation}\n{llm_prefix}"
    return thoughts
    

In [188]:
print(format_log_to_str(intermediate_steps))

I should use the get_text_length function to find the length of the text "dog".
Action: get_text_length
Action Input: "dog"
Observation:3
Thought:I should use the get_text_length function to find the length of the text "dog".
Action: get_text_length
Action Input: "dog"
Observation:3
Thought:I should use the get_text_length function to find the length of the text "dog".
Action: get_text_length
Action Input: "dog"
Observation:3
Thought:
